In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from itertools import islice
from IPython.display import clear_output
import pytry
import pickle

In [29]:
class PatternInterpolationTest(pytry.Trial):
    def params(self):
        self.param('number of dimensions', n_dims=2),
        #self.param('length of training pattern', len_train=10),
        self.param('number of epochs', n_epoch=1),
        self.param('seed', p_seed=1),
        
    def evaluate(self, param): #function to fill in missing data points by interpolating
        import nengo
        seed=param.p_seed
        dt = 0.001
        target_dt = 1.0/30
        D = param.n_dims
        theta = 15
        
        dec=np.load('%s_decoder.npy' % seed)
        pool_size_out=int(np.load('%s_node_pool.npy' % seed))
        
        goal_train_pca=np.load('%s_goal_train_pca.npy' % seed)
        noplay_train_pca=np.load('%s_noplay_train_pca.npy' % seed)
        
        goal_test_pca2=np.load('%s_goal_test_pca2.npy' % seed)
        noplay_test_pca2=np.load('%s_noplay_test_pca2.npy' % seed)
        aim_test_pca2=np.load('%s_aim_test_pca2.npy' % seed)
        
        goal_test_time=np.load('%s_goal_test_time.npy' % seed)
        noplay_test_time=np.load('%s_noplay_test_time.npy' % seed)
        aim_time=np.load('%s_aim_time.npy' % seed)
        
############ TESTING WITH 20% ############ 
        print('Testing with 20% Testing Data')
        a_goal=[]

        for j in range(1):#int(len(goal_test_pca2))):
            print('Testing Goal Pattern ', int(1+j))
            testing = np.vstack([goal_test_pca2[j]])

            test_net = nengo.Network(seed=seed)#param.seed)
            with test_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=pool_size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                goal_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(test_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[goal_test_result]

            a_goal.append(v)
            
        output_goal_test = {}
        for i in range(len(a_goal)):
            output_goal_test[i]={"time":goal_test_time[i], "output":a_goal[i]}

        for j in range(len(output_goal_test)):        
            key_j = 'output_goal_test{}'.format("%03d" % j)  
            output_goal_test[key_j] = output_goal_test.pop(j) 
            
############################   

        a_noplay=[]
        for j in range(1):#int(len(noplay_test_pca2))):
            print('Testing No Play Pattern ', int(1+j))
            testing = np.vstack([noplay_test_pca2[j]])

            test_net = nengo.Network(seed=seed)#param.seed)
            with test_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=pool_size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                noplay_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(test_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[noplay_test_result]

            a_noplay.append(v)
    
        #classify1 = np.isclose(v[:N], 1, atol=0.5)
        #classify2 = np.isclose(v[N:], -1, atol=0.5)
        #classify = np.append(classify1, classify2)
        #score_test = np.mean(classify)

        #classify_score[1]=score_test
            
        #test_timestamps = test_timestamps[0::2]

        output_noplay_test = {}
        for i in range(len(a_noplay)):
            output_noplay_test[i]={"time":noplay_test_time[i], "output":a_noplay[i]}

        for j in range(len(output_noplay_test)):        
            key_j = 'output_noplay_test{}'.format("%03d" % j)  
            output_noplay_test[key_j] = output_noplay_test.pop(j) 

############ TESTING WITH AIMLESS PATTERNS ############ 
        print('Testing with Aimless Data')

        print('Testing with Aimless Data')
        a_aim=[]

        for j in range(1):#len(aim_test_pca2)):
            print('Testing Aimless Pattern ', int(1+j))
            testing = np.vstack([aim_test_pca2[j]])

            aim_net = nengo.Network(seed=seed)#param.seed)
            with aim_net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))
                pool = nengo.Ensemble(n_neurons=3000, dimensions=pool_size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_stim = nengo.Probe(stim)

                result = nengo.Node(None, size_in=1)
                nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

                aim_test_result = nengo.Probe(result, sample_every = 0.1)


            test_sim = nengo.Simulator(aim_net)
            test_sim.run(int(len(testing))/30)

            v = test_sim.data[aim_test_result]

            a_aim.append(v)

        output_aimless = {}
        for i in range(len(a_aim)):
            output_aimless[i]={"time":aim_time[i], "output":a_aim[i]}

        for j in range(len(output_aimless)):        
            key_j = 'output_aimless{}'.format("%03d" % j)  
            output_aimless[key_j] = output_aimless.pop(j) 

############ SAVE DATA ############ 
       
        results = {**output_goal_test, **output_noplay_test, **output_aimless}

        return results  

In [30]:
PatternInterpolationTest().run(seed=0, p_seed=0, n_epoch=1, data_dir='debug', data_format="npz")

running PatternInterpolationTest#20190622-095219-1c805738
Testing with 20% Testing Data
Testing Goal Pattern  1


C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\signal\filter_design.py:1551: BadCoefficients: Badly conditioned filter coefficients (numerator): the results may be meaningless
  "results may be meaningless", BadCoefficients)
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengo\utils\numpy.py:79: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  v = a[inds]
C:\Users\mbartlett2\AppData\Local\Continuum\anaconda3\lib\site-packages\nengolib\signal\system.py:717: UserWarning: Filtering with non-SISO systems is an experimental feature that may not behave as expected.
  "expected.", UserWarning)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

...

<IPython.core.display.Javascript object>

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

<IPython.core.display.Javascript object>

seed = 0
n_dims = 2
n_epoch = 1
p_seed = 0
output_aimless000 = {'time': array(['1500029244', '1500029244', '1500029244', '1500029244',
       '1500029244', '1500029244', '1500029244', '1500029244',
       '1500029244', '1500029244', '1500029244', '1500029244',
       '1500029244', '1500029244', '1500029244', '1500029244',
       '1500029244', '1500029244', '1500029244', '1500029244',
       '1500029245', '1500029245', '1500029245', '1500029245',
       '1500029245', '1500029245', '1500029245', '1500029245',
       '1500029245', '1500029245', '1500029245', '1500029245',
       '1500029245', '1500029245', '1500029245', '1500029245',
       '1500029245', '1500029245', '1500029245', '1500029245',
       '1500029245', '1500029245', '1500029245', '1500029245',
       '1500029245', '1500029245', '1500029245', '1500029245',
       '1500029245', '1500029245', '1500029246', '1500029246',
       '1500029246', '1500029246', '1500029246', '1500029246',
       '1500029246', '1500029246', '1500029246

{'output_goal_test000': {'time': array(['1499685820', '1499685820', '1499685820', '1499685820',
         '1499685820', '1499685820', '1499685820', '1499685820',
         '1499685820', '1499685820', '1499685820', '1499685820',
         '1499685820', '1499685820', '1499685820', '1499685820',
         '1499685820', '1499685820', '1499685820', '1499685820',
         '1499685820', '1499685820', '1499685820', '1499685820',
         '1499685820', '1499685821', '1499685821', '1499685821',
         '1499685821', '1499685821', '1499685821', '1499685821',
         '1499685821', '1499685821', '1499685821', '1499685821',
         '1499685821', '1499685821', '1499685821', '1499685821',
         '1499685821', '1499685821', '1499685821', '1499685821',
         '1499685821', '1499685821', '1499685821', '1499685821',
         '1499685821', '1499685821', '1499685821', '1499685821',
         '1499685821', '1499685821', '1499685821', '1499685822',
         '1499685822', '1499685822', '1499685822', '1499685